In [1]:
import $file.common, common._
import $file.quel, quel._
import cats.data._, cats._, cats.implicits._
import doobie.implicits._
import doobie.util.fragment._
import doobie.util._, query._
import cats.evidence.Is

import $file.$     , common._

import $file.$   , quel._

import cats.data._, cats._, cats.implicits._

import doobie.implicits._

import doobie.util.fragment._

import doobie.util._, query._

import cats.evidence.Is

# Variation 6b. From QUEΛ to SQL 

The main purpose of this notebook is showing how can we generate efficient SQL queries from QUEΛ programs. Before that, however, we will do a simpler exercise to demonstrate the power of tagless-final APIs.

### A pretty-printer of QUEΛ programs

The question is this: can we actually write a pretty printer for the `largeCapitals` MTL-based query in [variance 4](Variance4.MTL.ipynb)?

`def largeCapitals[F[_]: Monad: FunctorFilter](implicit W: WorldRepo[F]): F[(String, String)]`

The answer is we can't. This pretty printer would require instantiating the corresponding APIs with a constant type constructor like `F[T] = String`. This could be certainly be done for the `WorldRepo` API, but not for `Monad` and `FunctorFilter`. Indeed, how could we implement the following signatures?

In [2]:
type ConstString[T] = String 

object StringMonad extends Monad[ConstString]{
    def flatMap[A, B](fa: String)(f: A => String): String = ???
    def pure[A](a: A): String = ???
    def tailRecM[A, B](a: A)(f: A => String): String = ???
}

object StringFunctorFilter extends FunctorFilter[ConstString]{
    def functor = ???
    def mapFilter[A, B](fa: String)(f: A => Option[B]): String = ???
}

defined type ConstString
defined object StringMonad
defined object StringFunctorFilter

The only thing we could implement in these instantiations is the `functor` field of `FunctorFilter`. The other components, we can't - not even the `pure[A]` component of `Monad` (we would need a `Show[A]` instance). For example, in the case of `flatMap`, how are we suppose to obtain a representation of the continuation `A => String`? The `A` is supposed to be computed from its argument `fa`, but `String` is no computation at all. What we are looking for is a _representation_ of that continuation, and tagless-final APIs fit the bill! 

Let's give a non-standard semantics for QUEΛ for pretty-printing:

In [3]:
abstract class StringRep[T] extends (String => Int => String)

object StringRep{
    
    def apply[T](f: String => Int => String): StringRep[T] = 
        new StringRep[T]{
            def apply(i: String): Int => String = f(i)
        }

    implicit object StringQUEΛ extends QUEΛ[StringRep]{
        // Base types

        def bool(b: Boolean): StringRep[Boolean] = 
            StringRep(_ => _ => s"Q.bool(${b.toString})")

        def int(i: Int): StringRep[Int] = 
            StringRep(_ => _ => s"Q.int(${i.toString})")

        def str(s: String): StringRep[String] = 
            StringRep(_ => _ => s"Q.str($s)")
                
        def ===(a1: StringRep[Int], a2: StringRep[Int]): StringRep[Boolean] = 
            StringRep(t => i => s"Q.===(${a1(t)(i)}, ${a2(t)(i)})")

        def >(i1: StringRep[Int], i2: StringRep[Int]): StringRep[Boolean] = 
            StringRep(t => i => s"Q.>(${i1(t)(i)}, ${i2(t)(i)})")

        // ADTs

        def tuple2[A, B](a: StringRep[A], b: StringRep[B]): StringRep[(A, B)] = 
            StringRep(t => i => s"""Q.tuple2(${a(t)(i)}, ${b(t)(i)})""")

        def none[A]: StringRep[Option[A]] = 
            StringRep(_ => _ => "none")

        def some[A](a: StringRep[A]): StringRep[Option[A]] = 
            StringRep(t => i => s"some(${a(t)(i)})")

        def exists[A](o: StringRep[Option[A]])(cond: StringRep[A] => StringRep[Boolean]): StringRep[Boolean] = 
            StringRep(t => i => s"""Q.exists(${o(t)(i)})(x$i => ${cond(_ => _ => s"x$i")(t)(i)})""")

        // Comprehensions

        def from[A, B](q: StringRep[List[A]])(f: StringRep[A] => StringRep[List[B]]): StringRep[List[B]] = 
            StringRep(t => i => 
                s"""|${t}Q.from(
                    |${q(t+"    ")(i)}
                    |${t}){ x$i => 
                    |${f(_ => _ => s"x$i")(t+"    ")(i+1)}
                    |${t}}""".stripMargin)

        def select[A](a: StringRep[A]): StringRep[List[A]] = 
            StringRep(t => i => t + s"Q.select(${a(t)(i)})")

        def where[A](cond: StringRep[Boolean])(q: StringRep[List[A]]): StringRep[List[A]] = 
            StringRep(t => i => 
                s"""|${t}Q.where(${cond(t)(i)})(
                    |${q(t+"    ")(i)}
                    |${t})""".stripMargin)
    }
    
    implicit object StringRepWorldModel extends WorldModel[StringRep]{
        // Cities
        
        def cityId(city: StringRep[City]): StringRep[Int] = 
            StringRep(t => i => s"W.cityId(${city(t)(i)})")

        def cityName(city: StringRep[City]): StringRep[String] = 
            StringRep(t => i => s"W.cityName(${city(t)(i)})")

        def cityCountry(city: StringRep[City]): StringRep[String] = 
            StringRep(t => i => s"W.cityCountry(${city(t)(i)})")

        def cityPopulation(city: StringRep[City]): StringRep[Int] = 
            StringRep(t => i => s"W.cityPopulation(${city(t)(i)})")

        // Countries
        
        def countryCode(country: StringRep[Country]): StringRep[String] = 
            StringRep(t => i => s"W.countryCode(${country(t)(i)})")

        def countryName(country: StringRep[Country]): StringRep[String] = 
            StringRep(t => i => s"W.countryName(${country(t)(i)})")

        def countryCapital(country: StringRep[Country]): StringRep[Option[Int]] = 
            StringRep(t => i => s"W.countryCapital(${country(t)(i)})")

        // World
        
        def allCountries: StringRep[List[Country]] = 
            StringRep(t => _ => t + s"W.allCountries")

        def allCities: StringRep[List[City]] = 
            StringRep(t => _ => t + s"W.allCities")
    }
}

defined class StringRep
defined object StringRep

The first argument in `String => Int => String` takes into account tabbing, and the second one is a counter used for giving names to variables (which appear in `exist` and `from`). But `from` is certainly similar to `flatMap`, how is it that we could implement it? Let's pay attention to the `from` signature:

`def from[A, B](q: Repr[List[A]])(f: Repr[A] => Repr[List[B]]): Repr[List[B]]`

As we can see, the continuation's type does not demand an `A` but a `Repr[A]`, i.e. we don't need a computation of `A`, but just a representation, which we can certainly provide. 

Let's obtain the pretty print representation of our `largeCapitals` query: 

In [4]:
import QUEΛSyntax._, WorldModelSyntax._

def largeCapitals[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- allCountries
        city <- allCities if country.capital.exists(_ === city.id)
        if city.population > 8000000
    } yield (city.name, country.name)

import QUEΛSyntax._, WorldModelSyntax._


defined function largeCapitals

In [5]:
println(largeCapitals[StringRep].apply("    ")(0))

    Q.from(
        W.allCountries
    ){ x0 => 
        Q.from(
            Q.from(
                Q.from(
                    W.allCities
                ){ x1 => 
                    Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                        Q.select(x1)
                    )
                }
            ){ x1 => 
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(x1)
                )
            }
        ){ x1 => 
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        }
    }


Note that there is no trace of the for-comprehension syntax, since the Scala preprocessor desugars these expressions into corresponding `flatMap`, `filter` and `map` calls. However, we can't see either these calls, where are they? They were also desugared, this time by our interpreter. In fact, tagless-final interpreters behave like preprocessors, and derived functions like `flatMap`, `map` and `filter` implemented in the `QUEΛSyntax` module, are actually much like macros!

Last, note that we obtain valid Scala code as we can manually check by copy-pasting the resulting `String` in the following signature:

In [6]:
def largeCapitalsPP[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.from(
        W.allCountries
    ){ x0 => 
        Q.from(
            Q.from(
                Q.from(
                    W.allCities
                ){ x1 => 
                    Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                        Q.select(x1)
                    )
                }
            ){ x1 => 
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(x1)
                )
            }
        ){ x1 => 
            Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
        }
    }

defined function largeCapitalsPP

In [7]:
largeCapitalsPP[StringRep].apply("")(0) == largeCapitals[StringRep].apply("")(0)

res6: Boolean = true

### Show me the SQL query!

We are ready now to face the real challenge: generate SQL expressions from `QUEΛ` queries. Informally, there seems to be a close correspondence between comprehensions and SQL: loosely speaking, `from` expressions generates a whole SQL statement; `where` expressions contribute to the SQL `WHERE` clause; and `select` expressions to the SQL `SELECT` one. Where do basic tables for the SQL `FROM` clause come from? The correspond to domain model expressions of type `List[A]`, for some type `A`. In the world domain model these are defined by `allCountries` and `allCities`. Ok, but if we analyse the pretty print result of `largeCapitals` we stumbled upon a mess of nested `from` expressions, that would certainly lead to nested subqueries, if at all correct. Right, let's look to the following _normalized_ query instead, in order to see the correspondence of `QUEΛ` to `SQL` more directly:

In [8]:
def largeCapitalsNormalized[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.from(W.allCountries){ x0 => 
        Q.from(W.allCities){ x1 => 
            Q.where(W.countryCapital(x0).exists(_ === W.cityId(x1)))(
                Q.where(W.cityPopulation(x1) > Q.int(8000000))(
                    Q.select((W.cityName(x1), W.countryName(x0)))
                )
            )
        }
    }

defined function largeCapitalsNormalized

In [9]:
println(largeCapitalsNormalized[StringRep].apply("    ")(0))

    Q.from(
        W.allCountries
    ){ x0 => 
        Q.from(
            W.allCities
        ){ x1 => 
            Q.where(Q.exists(W.countryCapital(x0))(x2 => Q.===(x2, W.cityId(x1))))(
                Q.where(Q.>(W.cityPopulation(x1), Q.int(8000000)))(
                    Q.select(Q.tuple2(W.cityName(x1), W.countryName(x0)))
                )
            )
        }
    }


This query is said to be in _normal form_: very informally, we can understand this query as a linearly ordered sequence of `from` expressions, followed by `where` expressions, and ended by the `select` one. This can be easily translated to SQL. But the problem is, then, how to transform any arbitrary, possibly messy, `QUEΛ` expression into its normal form. There are two major alternatives: we may implement a re-writing interpreter that performs syntactic transformations until the normal form is found; or we may perform the bulk of the normalization process at the semantic level, coming back to the syntactic level to generate the normal form as a simple step. This second strategy is called _normalization by evaluation_ and it's explained in the following [paper](http://okmij.org/ftp/meta-programming/#SQUR) by Oleg and Tatsuya:

![](images/maintainingpaper.png)

In what follows, we will apply the normalization-by-evaluation strategy introduced in this paper. The only minor difference is that we won't come back to the syntactic level to obtain the normalized query, but we will generate SQL directly from the chosen semantic representation (as it seems to be done in the [implementation](http://okmij.org/ftp/meta-programming/Sqr/) of the paper itself). Well, talk of the devil, here it is!

In [49]:
sealed abstract class DoobieRep[T]
case class SQLVar[A](fragment: Fragment) extends DoobieRep[A] // TBD: This makes pattern matching unsafe
case class SQLInt(fragment: Fragment) extends DoobieRep[Int]
case class SQLBoolean(fragment: Fragment) extends DoobieRep[Boolean]
case class SQLString(fragment: Fragment) extends DoobieRep[String]
case class SQLOption[A](rep: Option[DoobieRep[A]]) extends DoobieRep[Option[A]]
case class SQLTuple[A, B](_1: DoobieRep[A], _2: DoobieRep[B]) extends DoobieRep[(A, B)]
case class SQLList[A](
    from: List[DoobieRep.Table], 
    where: List[DoobieRep[Boolean]], 
    select: DoobieRep[A]) extends DoobieRep[List[A]]

object DoobieRep{
    case class Table(name: String, varName: String)
}

defined class DoobieRep
defined class SQLVar
defined class SQLInt
defined class SQLBoolean
defined class SQLString
defined class SQLOption
defined class SQLTuple
defined class SQLList
defined object DoobieRep

Wow, the `Id[_]` and `StringRep[_]` representations were more simple! Let's not despair, this is certainly more complex, but the complexity will hopefully vanish if we note that the semantic domain is a GADT whose type discriminators correspond to the types of `QUEΛ` expressions: the base types `Integer`, `Boolean` and `String`, the ADTs `Tuple2` and `Option`, and the multiset `List`. And the representations for base types and ADTs are simple: the representation of base types is a doobie `Fragment`, i.e. a fragment of SQL expression; the representation of a tuple is a tuple of representations, and the representation of an optional expression is an optional representation. The representation of `List` is more involved, but it simply describes the basic components of a complete SQL query: the components of the `FROM` clause (a sequence of tables), the `WHERE` clause (a sequence of boolean representations) and the `SELECT` clause (a representation of type `A`). Last, note that there is an special case in the GADT for variables of a generic type `A`.

Let's recall our goal: we want to obtain a simple SQL select statement for `QUEΛ` expressions of type `List`, without nested subqueries in the `FROM` clause. Let's see how we can certainly generate this kind of SQL query from the above representation, and let's leave for later the task of figuring out how to generate that from `QUEΛ` and `WorldModel` expressions. 

In [50]:
implicit class FragmentDoobieRep[T](d: DoobieRep[T]){ 
    
    val fragment: Fragment = d match {
        case d: SQLVar[_]      => d.fragment
        case d: SQLInt         => d.fragment
        case d: SQLBoolean     => d.fragment
        case d: SQLString      => d.fragment
        case d: SQLOption[_]   => d.rep.fold(fr0"")(_.fragment)
        case d: SQLTuple[_, _] => d._1.fragment ++ fr0", " ++ d._2.fragment
        case d: SQLList[_]     => sqlListFragment(d)
    }
    
    private def sqlListFragment[A](d: SQLList[A]): Fragment = {
        
        def tableFragment(table: DoobieRep.Table): Fragment = 
            Fragment.const(table.name) ++ Fragment.const0(table.varName)
        
        val fromFr: List[Fragment] = 
            if (d.from.isEmpty) List() 
            else List(fr"from" ++ d.from.map(tableFragment).mkFragment(fr0",", false))
        
        val whereFr: List[Fragment] = 
            if (d.where.isEmpty) List() 
            else List(fr"where" ++ d.where.map(_.fragment).mkFragment(fr0"and"))

        (List(fr"select" ++ d.select.fragment) ++ fromFr ++ whereFr)
            .mkFragment(fr0" ", false, false)
    }
}

defined class FragmentDoobieRep

As we see, the fragment generation for base and ADTs is straightforward. And the fragment corresponding to list representations is a simple SQL select statement with no nested subqueries in the from clause (since this clause is made of atomic _tables_). In order to obtain a full-fledged doobie query, the only thing needed is to summon the `Read` instance for the record type `A`: 

In [51]:
implicit class ListDoobieRepToQuery0[A: Read](d: DoobieRep[List[A]]){
    def query: Query0[A] = 
        d.fragment.query[A](Read[A])
}

defined class ListDoobieRepToQuery0

Ok, we see that this representation will serve our purpose, but, how do we generate it? Clearly, we need to give instances of the `WorldModel` and `QUEΛ` APIs, which account for a new non-standard semantics of our DSL. Let's start with the former:

In [68]:
object DoobieRepWorldModel extends WorldModel[DoobieRep]{
    import ModelConstructors._
    
    // Cities

    def cityId(city: DoobieRep[City]): DoobieRep[Int] = 
        intColumn(city, "id")

    def cityName(city: DoobieRep[City]): DoobieRep[String] = 
        stringColumn(city, "name")

    def cityCountry(city: DoobieRep[City]): DoobieRep[String] = 
        stringColumn(city, "country")

    def cityPopulation(city: DoobieRep[City]): DoobieRep[Int] = 
        intColumn(city, "population")

    // Countries

    def countryCode(country: DoobieRep[Country]): DoobieRep[String] = 
        stringColumn(country, "code")

    def countryName(country: DoobieRep[Country]): DoobieRep[String] = 
        stringColumn(country, "name")

    def countryCapital(country: DoobieRep[Country]): DoobieRep[Option[Int]] = 
        nullableColumn(intColumn(country, "capital"))

    // World

    def allCountries: DoobieRep[List[Country]] =
        table(DoobieRep.Table("country", "x0"))

    def allCities: DoobieRep[List[City]] = 
        table(DoobieRep.Table("city", "x1"))
}

object ModelConstructors{ 

    def table[A](table: DoobieRep.Table): SQLList[A] = 
        SQLList(List(table), List(), SQLVar(Fragment.const0(table.varName)))
    
    def intColumn[A](d: DoobieRep[A], field: String): DoobieRep[Int] = 
        SQLInt(d.fragment ++ Fragment.const0(s".$field"))

    def stringColumn[A](d: DoobieRep[A], field: String): DoobieRep[String] = 
        SQLString(d.fragment ++ Fragment.const0(s".$field"))

    def boolColumn[A](d: DoobieRep[A], field: String): DoobieRep[Boolean] = 
        SQLBoolean(d.fragment ++ Fragment.const0(s".field"))
    
    def nullableColumn[A](d: DoobieRep[A]): DoobieRep[Option[A]] = 
        SQLOption(Some(d))
}

defined object DoobieRepWorldModel
defined object ModelConstructors

This instance defines the functional-relational mapping between the tagless-final specification and the relational schema. The general utilities in the `ModelConstructors` module help us to define this mapping more easily. The `table` constructor for a table `table_name t` creates a SQL select statement of the form `SELECT t.* FROM table_name t`. The `*Column` constructors creates representations based on column references for the different base types. 

Let's move now to the general combinators of `QUEΛ`:

In [69]:
object DoobieRepQUEΛ extends QUEΛ[DoobieRep]{
    // base types

    def bool(b: Boolean): DoobieRep[Boolean] = 
        SQLBoolean(Fragment.const(b.toString))

    def int(i: Int): DoobieRep[Int] = 
        SQLInt(Fragment.const0(i.toString))

    def str(s: String): DoobieRep[String] = 
        SQLString(Fragment.const(s))

    def ===(a1: DoobieRep[Int], a2: DoobieRep[Int]): DoobieRep[Boolean] = 
        SQLBoolean(a1.fragment ++ fr" =" ++ a2.fragment)

    def >(i1: DoobieRep[Int], i2: DoobieRep[Int]): DoobieRep[Boolean] = 
        SQLBoolean(i1.fragment ++ fr0" > " ++ i2.fragment)

    // ADTs

    def tuple2[A, B](a: DoobieRep[A], b: DoobieRep[B]): DoobieRep[(A, B)] = 
        SQLTuple(a, b) 

    def none[A]: DoobieRep[Option[A]] = 
        SQLOption(None)

    def some[A](a: DoobieRep[A]): DoobieRep[Option[A]] = 
        SQLOption(Some(a))

    def exists[A](o: DoobieRep[Option[A]])(cond: DoobieRep[A] => DoobieRep[Boolean]): DoobieRep[Boolean] = {
        val SQLOption(maybeA) = o
        maybeA.fold(bool(false))(cond)
    }

    // Comprehensions

    def from[A, B](q: DoobieRep[List[A]])(f: DoobieRep[A] => DoobieRep[List[B]]): DoobieRep[List[B]] = {
        val SQLList(from1, where1, select1) = q
        val SQLList(from2, where2, select2) = f(select1)
        SQLList(from1 ++ from2, where1 ++ where2, select2)
    }

    def where[A](cond: DoobieRep[Boolean])(q: DoobieRep[List[A]]): DoobieRep[List[A]] = {
        val SQLList(from, where, select) = q
        SQLList(from, cond :: where, select)
    }        
    
    def select[A](a: DoobieRep[A]): DoobieRep[List[A]] = 
        SQLList(List(), List(), a)
}

defined object DoobieRepQUEΛ

The semantics for base types and ADTs are given in terms of SQL fragments representing value expressions of different types: constants and operator invocations, in particular. The crux of the semantics is in the implementation of the `from` combinator, since it's here where nested subqueries are flatten. Let's obtain the SQL fragment representation for our `largeCapitals` query:

In [75]:
largeCapitals(DoobieRepQUEΛ, DoobieRepWorldModel).fragment

res74: Fragment = Fragment("select x1.name, x0.name from country x0, city x1 where x0.capital = x1.id and x1.population > 8000000")

It works! Let's execute it:

In [76]:
largeCapitals(DoobieRepQUEΛ, DoobieRepWorldModel).query.to[List].transact(xa).unsafeRunSync

res75: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

### Does modularity affects the generation?

Let's check the generated SQL query for a more modular implementation:

In [81]:
def city[Repr[_]: QUEΛ: WorldModel](cityId: Repr[Option[Int]]): Repr[List[City]] = 
    for {
        c <- allCities if cityId.exists{ _ === c.id }
    } yield c

def largeCity[Repr[_]: QUEΛ: WorldModel](cityId: Repr[Option[Int]]): Repr[List[City]] = 
    for {
        c <- city(cityId)
        if c.population > 8000000
    } yield c

def largeCapitalsModular[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- allCountries
        city <- largeCity(country.capital)
    } yield (city.name, country.name)

defined function city
defined function largeCity
defined function largeCapitalsModular

In [85]:
largeCapitalsModular(DoobieRepQUEΛ, DoobieRepWorldModel).fragment.toString == 
largeCapitals(DoobieRepQUEΛ, DoobieRepWorldModel).fragment.toString

res84: Boolean = true

The exact same query!